In [2]:
# 测试评测能力
from eval import eval

eval('/root/project/Prompt-BERT/result/unsup-bert_s42/')

{'avg_scores': {'STS12': 0.15847410361933553,
  'STS13': 0.2293880825819684,
  'STS14': 0.15145239504982302,
  'STS15': 0.21778545913447578,
  'STS16': 0.3346207631216493,
  'STSBenchmark': 0.1620628433967177,
  'SICKRelatedness': 0.32394333064872427,
  'avg': 0.2253895682218134},
 'task_scores': {'STS12': [0.15847410361933553,
   0.15847410361933553,
   0.15847410361933553],
  'STS13': [0.2293880825819684, 0.2293880825819684, 0.2293880825819684],
  'STS14': [0.15145239504982302, 0.15145239504982302, 0.15145239504982302],
  'STS15': [0.21778545913447578, 0.21778545913447578, 0.21778545913447578],
  'STS16': [0.33462076312164923, 0.33462076312164923, 0.33462076312164923],
  'STSBenchmark': [0.1620628433967177, 0.1620628433967177, 0.1620628433967177],
  'SICKRelatedness': [0.32394333064872427,
   0.32394333064872427,
   0.32394333064872427],
  'avg': [0.2253895682218134, 0.2253895682218134, 0.2253895682218134]}}